In [79]:
import os
import glob
import pandas as pd
import numpy as np

# Read the two csv files
data1 = pd.read_csv('DPQ_I.csv')
data2 = pd.read_csv('DEMO_I.csv')

# Maybe start by merging the two files by SEQN column
# Merge the two files with the SEQN column
merged_data = pd.merge(data1, data2 , how='left', on='SEQN')

# Only include the columns below in the DEMO_I spreadsheet
# Race, Gender, Age, Education, Marital Status, and PIR
merged_data = merged_data[['SEQN', 'RIDRETH1', 'RIAGENDR', 'RIDAGEYR', 'DMDEDUC3', 'DMDEDUC2', 'DMDMARTL', 'INDFMPIR',
                           'DPQ010','DPQ020','DPQ030','DPQ040','DPQ050','DPQ060','DPQ070','DPQ080','DPQ090']]

# Explore the dataset
print(merged_data.info(), '\n')
merged_data.shape[0]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5735 entries, 0 to 5734
Data columns (total 17 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   SEQN      5735 non-null   float64
 1   RIDRETH1  5735 non-null   float64
 2   RIAGENDR  5735 non-null   float64
 3   RIDAGEYR  5735 non-null   float64
 4   DMDEDUC3  261 non-null    float64
 5   DMDEDUC2  5474 non-null   float64
 6   DMDMARTL  5474 non-null   float64
 7   INDFMPIR  5134 non-null   float64
 8   DPQ010    5164 non-null   float64
 9   DPQ020    5164 non-null   float64
 10  DPQ030    5164 non-null   float64
 11  DPQ040    5162 non-null   float64
 12  DPQ050    5162 non-null   float64
 13  DPQ060    5162 non-null   float64
 14  DPQ070    5161 non-null   float64
 15  DPQ080    5161 non-null   float64
 16  DPQ090    5161 non-null   float64
dtypes: float64(17)
memory usage: 806.5 KB
None 



5735

In [80]:
# Data cleaning for the depression columns 

# Drop the rows that have nan for questions 1-9
merged_data = merged_data[merged_data['DPQ010'].notna()]

# Add the values of each row in the depression spreadsheet (DPQI_)
merged_data['SCORE'] = merged_data['DPQ010'] + merged_data['DPQ020'] + merged_data['DPQ030'] + merged_data['DPQ040'] + merged_data['DPQ050'] + merged_data['DPQ060'] + merged_data['DPQ070'] + merged_data['DPQ080'] + merged_data['DPQ090']

# If a participant had total PHQ-9 ≥ 10, the person is considered having depression.
for index, row in merged_data.iterrows():
    if merged_data.at[index,'SCORE'] >= 10:
        merged_data.at[index,'DEPRESSED'] = 1
    else:
        merged_data.at[index,'DEPRESSED'] = 0

# pandas count distinct values in column
print(merged_data['DEPRESSED'].value_counts())
print(merged_data['DMDEDUC3'].value_counts())

0.0    4731
1.0     433
Name: DEPRESSED, dtype: int64
13.0    102
15.0     54
11.0     50
12.0     21
10.0      9
14.0      5
66.0      3
9.0       2
Name: DMDEDUC3, dtype: int64


In [81]:
# Data cleaning for demographics spreadsheet (DEMO_I)

# merged_data = merged_data.dropna(subset=['RIDRETH1', 'RIAGENDR', 'RIDAGEYR', 'DMDEDUC3', 'DMDEDUC2', 'DMDMARTL', 'INDFMPIR'])

# Only include participants with an age 18 or older 
merged_data = merged_data[merged_data['RIDAGEYR'] >= 18]  

# Only include participants with an education level known for DMDEDUC2
merged_data = merged_data[merged_data['DMDEDUC2'] != 9] 

for index, row in merged_data.iterrows():
    
    # Divide the data into two age groups
    # 1.) Younger: 18-55 
    # 2.) Older: 56+
    if merged_data.at[index,'RIDAGEYR'] <= 55:
        merged_data.at[index,'AGEGRP'] = 1
    else:
        merged_data.at[index,'AGEGRP'] = 2

    # Divide the data into maritial status
    # 1.) Married: 1
    # 2.) Widowed/Divorced/Separated: 2, 3, 4 (combined)
    # 3.) Never Married: 5
    # 4.) Living with Partner: 6
    if merged_data.at[index,'DMDMARTL'] == 1.0:
        merged_data.at[index,'MARITALSTATUS'] = 1
    elif merged_data.at[index,'DMDMARTL'] == 2.0 or merged_data.at[index,'DMDMARTL'] == 3.0 or merged_data.at[index,'DMDMARTL'] == 4.0:
        merged_data.at[index,'MARITALSTATUS'] = 2
    elif merged_data.at[index,'DMDMARTL'] == 5.0:
        merged_data.at[index,'MARITALSTATUS'] = 3
    else:
        merged_data.at[index,'MARITALSTATUS'] = 4

    # Divide the data into PIR (SES)
    # 1.) Low: 0-1.36
    # 2.) Medium: 1.37-4.99
    # 3.) High: 5.00+
    if merged_data.at[index,'INDFMPIR'] >= 5.00:
        merged_data.at[index,'PIR'] = 3
    elif merged_data.at[index,'INDFMPIR'] <= 4.99 and merged_data.at[index,'INDFMPIR'] >= 1.37:
        merged_data.at[index,'PIR'] = 2
    else:
        merged_data.at[index,'PIR'] = 1

    # Divide the data by education level
    # 1.) Up to 11th grade: DMDEDUC3 = 0-11 & 66; DMDEDUC2 = 1-2
    # 2.) High School or GED: DMDEDUC3 = 12-14; DMDEDUC2 = 3
    # 3.) Some College or Associate Degree: DMDEDUC3 = 15; DMDEDUC2 = 4
    # 4.) College Graduate or Above: DMDEDUC2 = 5
    if merged_data.at[index,'DMDEDUC3'] <= 11 or merged_data.at[index,'DMDEDUC3'] == 66 or merged_data.at[index,'DMDEDUC2'] <= 2:
        merged_data.at[index,'EDUCATION'] = 1
    elif merged_data.at[index,'DMDEDUC3'] >= 12 or merged_data.at[index,'DMDEDUC3'] <= 13 or merged_data.at[index,'DMDEDUC3'] <= 14 or merged_data.at[index,'DMDEDUC2'] <= 3:
        merged_data.at[index,'EDUCATION'] = 2
    elif merged_data.at[index,'DMDEDUC3'] == 15 or merged_data.at[index,'DMDEDUC2'] == 4:
        merged_data.at[index,'EDUCATION'] = 3
    else:
        merged_data.at[index,'EDUCATION'] = 4
    
    # Divide the data by race/ethnicity
    # 1.) Mexican American: 1-2
    # 2.) White: 3
    # 3.) African-American: 4
    # 4.) Other Race: 5
    if merged_data.at[index,'RIDRETH1'] == 1 or merged_data.at[index,'RIDRETH1'] == 2:
        merged_data.at[index,'RACE'] = 1
    elif merged_data.at[index,'RIDRETH1'] == 3:
        merged_data.at[index,'RACE'] = 2
    elif merged_data.at[index,'RIDRETH1'] == 4:
        merged_data.at[index,'RACE'] = 3
    else:
        merged_data.at[index,'RACE'] = 4

In [82]:
# pandas count distinct values in column
merged_data = merged_data.rename(columns={'RIAGENDR': 'GENDER'})

print(merged_data['DEPRESSED'].value_counts())
print(merged_data['AGEGRP'].value_counts())
print(merged_data['MARITALSTATUS'].value_counts())
print(merged_data['PIR'].value_counts())
print(merged_data['GENDER'].value_counts())
print(merged_data['EDUCATION'].value_counts())
print(merged_data['RACE'].value_counts())

print('\nVariables, n=', merged_data.shape[0])

0.0    4730
1.0     432
Name: DEPRESSED, dtype: int64
1.0    3185
2.0    1977
Name: AGEGRP, dtype: int64
1.0    2474
2.0    1063
3.0     902
4.0     723
Name: MARITALSTATUS, dtype: int64
2.0    2300
1.0    2122
3.0     740
Name: PIR, dtype: int64
2.0    2639
1.0    2523
Name: GENDER, dtype: int64
3.0    1476
2.0    1269
4.0    1217
1.0    1200
Name: EDUCATION, dtype: int64
2.0    1717
1.0    1596
3.0    1105
4.0     744
Name: RACE, dtype: int64

Variables, n= 5162


In [83]:
# Export the merged files to a csv
header = ['SEQN', 'DEPRESSED', 'AGEGRP', 'MARITALSTATUS', 'PIR', 'GENDER', 'RACE', 'EDUCATION']
merged_data.to_csv('./mergeddata.csv', columns=header)

In [84]:
merged_data.shape[0]

5162